# HUQ data processing

In [1]:
import petl as etl
from utils.paths import HUQ, SITE

In [2]:
ticketing = (
    etl
    .fromcsv(HUQ / 'mastercard.csv')
    .convertnumbers()
    .convert('datestamp', etl.dateparser('%Y-%m-%d'))
)

In [3]:
ticketing

datestamp,variable,value
2024-12-01,average_account_count,215
2024-12-01,average_spend_amount,40
2024-12-01,average_ticket,38
2024-12-01,transaction_amount,142
2024-12-01,transaction_count,223


In [4]:
OUTPUT = SITE / 'experiments/huq/_data'
OUTPUT.mkdir(exist_ok=True, parents=True)

In [5]:
(
    ticketing
    .selecteq('variable', 'average_account_count')
    .cutout('variable')
    .tocsv(OUTPUT / 'footfall_proxy.csv')
)

In [6]:
(
    ticketing
    .selecteq('variable', 'transaction_amount')
    .cutout('variable')
    .tocsv(OUTPUT / 'spend_index.csv')
)

In [ ]:
footfall = (
    etl
    .fromcsv(HUQ / 'footfall.csv')
    .cut('datestamp', 'centre_name', 'estimated_actual_footfall_rolling')
    .rename(2, 'value')
    .convertnumbers()
    .convert('datestamp', etl.dateparser('%Y-%m-%d'))
    .cache()
)

In [8]:
import re
for k, t in footfall.facet('centre_name').items():
    filename = re.sub(r'[\s-]+', '_', k.lower()) + '.csv'
    t.tocsv(OUTPUT / 'footfall' / filename)